In [1]:
import pandas
pandas.set_option('display.max_colwidth', -1)
pandas.set_option('max_seq_items', None)
pandas.set_option('display.max_rows', 120)

# Tokens and analysis

This demo is based on 123844 tokens with 205005 analysis.

We have a file of tokens with analyses where every line has json encoded form
```
token, [(lemma, root, root_tokens, ending, clitic, partofspeech, form), ...]
```
For example:
```
["Ühtedel", [["üks", "üks", ["üks"], "del", "", "N", "pl ad"], ["üks", "üks", ["üks"], "del", "", "P", "pl ad"]]]
```

## Create a dict analysis_examples
```
{(cap, dash, eq, ending, clitic, partofspeech, form): token, ...}
```
where 

        cap == True, if token starts with a capital letter,
        cap == False, if token starts with a lower case letter,
        cap == None, if token does not start with a letter,
        dash == True, if '-' in the root,
        dast == False if '-' not in the root,
        eq == True, if '=' in the root,
        eq == False if '=' not in the root.

In [2]:
from collections import Counter
from tokens_analysis import yield_tokens_analysis
file='../corpora/tokens_analysis.json'

analysis_examples = {}
analysis_counter = Counter()
for token, analysis in yield_tokens_analysis(file):
    if token[0].islower():
        cap = False
    elif token[0].isupper():
        cap = True
    elif not token[0].isalpha():
        cap = None
    for a in analysis:
        key = (cap, 
               '-' in a['root'], 
               '=' in a['root'], 
               a['ending'], 
               a['clitic'], 
               a['partofspeech'], 
               a['form'])
        analysis_examples[key] = token
        analysis_counter[key] += 1

print('20 most common analysis out of', len(analysis_examples), '\n')
for a, n in analysis_counter.most_common(20):
    print('{:<50} {:<20} {:>10}'.format(str(a), analysis_examples[a], n))

20 most common analysis out of 1658 

(True, False, False, '0', '', 'H', 'sg n')         Προϊόντα                  18159
(True, False, False, '0', '', 'H', 'sg g')         Εcu                       13890
(False, False, False, '0', '', 'S', 'sg n')        τόνο                      12402
(False, False, False, '0', '', 'S', 'sg g')        žürii                     12330
(False, False, False, '0', '', 'S', 'sg p')        žiletitera                 5039
(False, False, False, 'd', '', 'S', 'pl n')        žüriid                     4663
(False, False, False, '0', '', 'S', 'adt')         želatiini                  4043
(True, False, False, '0', '', 'S', 'sg n')         Žürii                      3654
(False, False, False, 't', '', 'S', 'sg p')        žiirot                     3427
(True, False, False, '0', '', 'S', 'sg g')         Žürii                      3353
(False, False, False, 's', '', 'S', 'sg in')       žanris                     3112
(False, False, False, 'st', '', 'S', 'sg el')    

## Text of tokens that have pairwise different analysis

Maybe we can use this text for testing syntax_preprocessing efficiently.

In [3]:
' '.join(list(analysis_examples.values())[:100])

'väliseesti luks-pesakeseks MV-te 5-ukseline ühtesid enamasilbilisil Ütlen Vastastele üksikasjadeni ütleksime suurimal vanematega üldhariduslikes Sobigi Ülikoolide käsile žiletitera tegeldavat 7.-klassilise Läänemeremaadeni jõulu-Lennartiks kergejõustiku-meistrivõistlustelt 18-50-aastaseid Ühelt – uusimale Ühis- öelnud-kirjutanud üleliidulistega On kolmaski Olümpiatule 40-tunnilise Ühendriikidelt samm-sammulise pisimategi žsamaaliidul demograafilisse vormel-üks sind ühtegi üürisid Üksikuid ühtteist polnud Ei üleülikooliliste värskeima teistelegi võikski ütlustes õmblejatele-meistritele 8-9aastastele Žürii oDežille ühele 20-ventiilise põlvini tele-eetrisse teisegi lähedalegi Takistustena Rahvarinde Rikkaimate demokraatlikena vanadest olemaski vanade- Täpsemat ütlemagi Vähemalt pre-orderiks üks-kaks viide mida Tugevaimaks vanus Üle-eelmisel projekteeritavates 724-käideldavust -sada i-mitmuseja teisse Üksikasjalisemat Teistes 90-ndail ühtede Üheksandat Teistest 60seks ülikoolides teisegi 

## Pronouns in corpora

Find all tokens that are marked as pronouns (partofspeech P). 

### Group the pronouns by regexes from \_pronConversions
The class 'no_search_match' is for pronouns that don't match any regex.

In [4]:
_pronConversions = ( ("emb\+.*",             ("det",)),
                     ("enda\+.*",            ("pos", "refl")),
                     ("enese\+.*",           ("pos", "refl")),
                     ("eikeegi.*",           ("indef",)),
                     ("eimiski.*",           ("indef",)),
                     ("emb-kumb.*",          ("det",)),
                     ("esimene.*",           ("dem",)),
                     ("iga\+.*",             ("det",)),
                     ("iga_sugune.*",        ("indef",)),
                     ("iga_.ks\+.*",         ("det",)),
                     ("ise\+.*",             ("pos", "det", "refl")),
                     ("ise_enese.*",         ("refl",)),
                     ("ise_sugune.*",        ("dem",)),
                     ("keegi.*",             ("indef",)),
                     ("kes.*",               ("inter rel",)),
                     ("kumb\+.*",            ("rel",)),
                     ("kumbki.*",            ("det",)),
                     ("kõik.*",              ("det",)),
                     ("k.ik.*",              ("det",)),
                     ("meie_sugune.*",       ("dem",)),
                     ("meie_taoline.*",      ("dem",)),
                     ("mihuke\+.*",          ("inter rel",)),
                     ("mihukene\+.*",        ("inter rel",)),
                     ("mille_taoline.*",     ("dem",)),
                     ("milli=?ne.*",         ("rel",)),
                     ("mina\+.*",            ("pers ps1",)),
                     (" ma\+.*",             ("pers ps1",)),
                     ("mina=?kene\+.*",      ("dem",)),
                     ("mina=?ke\+.*",        ("dem",)),
                     ("mingi\+.*",           ("indef",)),
                     ("mingi_sugune.*",      ("indef",)),
                     ("minu_sugune.*",       ("dem",)),
                     ("minu_taoline.*",      ("dem",)),
                     ("miski.*",             ("indef",)),
                     ("mis\+.*",             ("inter rel",)),
                     ("mis_sugune.*",        ("inter rel",)),
                     ("miski\+.*",           ("inter rel",)),
                     ("miski_sugune.*",      ("inter rel",)),
                     ("misu=?ke(ne)?\+.*",   ("dem",)),
                     ("mitme_sugune.*",      ("indef",)),
                     ("mitme_taoline.*",     ("indef",)),
                     ("mitmendik\+.*",       ("inter rel",)),
                     ("mitmes\+.*",          ("inter rel", "indef")),
                     ("mi=?tu.*",            ("indef",)),
                     ("miuke(ne)?\+.*",      ("inter rel",)),
                     ("muist\+.*",           ("indef",)),
                     ("muu.*",               ("indef",)),
                     ("m.lema.*",            ("det",)),
                     ("m.ne_sugune\+.*",     ("indef",)),
                     ("m.ni\+.*",            ("indef",)),
                     ("m.ningane\+.*",       ("indef",)),
                     ("m.ningas.*",          ("indef",)),
                     ("m.herdune\+.*",       ("indef", "rel")),
                     ("määntne\+.*",         ("dem",)),
                     ("na_sugune.*",         ("dem",)),
                     ("nende_sugune.*",      ("dem",)),
                     ("nende_taoline.*",     ("dem",)),
                     ("nihuke(ne)?\+.*",     ("dem",)),
                     ("nii_mi=?tu\+.*",      ("indef", "inter rel")),
                     ("nii_sugune.*",        ("dem",)),
                     ("niisama_sugune.*",    ("dem",)),
                     ("nii?su=?ke(ne)?\+.*", ("dem",)),
                     ("niuke(ne)?\+.*",      ("dem",)),
                     ("oma\+.*",             ("pos", "det", "refl")),
                     ("oma_enese\+.*",       ("pos",)),
                     ("oma_sugune\+.*",      ("dem",)),
                     ("oma_taoline\+.*",     ("dem",)),
                     ("palju.*",             ("indef",)),
                     ("sama\+.*",            ("dem",)),
                     ("sama_sugune\+.*",     ("dem",)),
                     ("sama_taoline\+.*",    ("dem",)),
                     ("samune\+.*",          ("dem",)),
                     ("see\+.*",             ("dem",)),
                     ("see_sama\+.*",        ("dem",)),
                     ("see_sam[au]ne\+.*",   ("dem",)),
                     ("see_sinane\+.*",      ("dem",)),
                     ("see_sugune\+.*",      ("dem",)),
                     ("selle_taoline\+.*",   ("dem",)),
                     ("selli=?ne\+.*",       ("dem",)),
                     ("setu\+.*",            ("indef",)),
                     ("setmes\+.*",          ("indef",)),
                     ("sihuke\+.*",          ("dem",)),
                     ("sina\+.*",            ("pers ps2",)),
                     (" sa\+.*",             ("pers ps2",)),
                     ("sinu_sugune\+.*",     ("dem",)),
                     ("sinu_taoline\+.*",    ("dem",)),
                     ("siuke(ne)?\+.*",      ("dem",)),
                     ("säherdune\+.*",       ("dem",)),
                     ("s.herdune\+.*",       ("dem",)),
                     ("säärane\+.*",         ("dem",)),
                     ("s..rane\+.*",         ("dem",)),
                     ("taoline\+.*",         ("dem",)),
                     ("teie_sugune\+.*",     ("dem",)),
                     ("teie_taoline\+.*",    ("dem",)),
                     ("teine\+.*",           ("dem",)),
                     ("teine_teise\+.*",     ("rec",)),
                     ("teist?_sugune\+.*",   ("dem",)),
                     ("tema\+.*",            ("pers ps3",)),
                     (" ta\+.*",             ("pers ps3",)),
                     ("temake(ne)?\+.*",     ("pers ps3",)),
                     ("tema_sugune\+.*",     ("dem",)),
                     ("tema_taoline\+.*",    ("dem",)),
                     ("too\+.*",             ("dem",)),
                     ("too_sama\+.*",        ("dem",)),
                     ("üks.*",               ("dem", "indef")),
                     (".ks.*",               ("dem", "indef")),
                     ("ükski.*",             ("dem", "indef")),
                     (".kski.*",             ("dem", "indef")),
                     ("üks_teise.*",         ("rec", "indef")),
                     (".ks_teise.*",         ("rec",))
)

import regex

search = regex.search
match = regex.match
pronoun_examples = {}
for a, b in _pronConversions:
    pronoun_examples[a] = []
no_search_match_examples = []

for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['partofspeech'] != 'P':
            continue
        root_ec = ''.join((a['root'], '+', a['ending'], a['clitic']))
        search_match = False
        for pattern, l in pronoun_examples.items():
            if search(pattern, root_ec):
                # kas search või match? "enese" vs "iseenese"
                # kas token asemel võiks olla hoopis token.lower() või root või lemma (ja l oleks siis list asemel set)?
                l.append(token)
                search_match = True
        if not search_match:
            no_search_match_examples.append(token)
pronoun_examples['no_search_match'] = no_search_match_examples

data = [{'pattern': key, 'examples': sorted(value)} for key, value in sorted(pronoun_examples.items())]
pandas.DataFrame.from_records(data, columns=['pattern', 'examples'])

,pattern,examples
0,ma\+.*,[]
1,sa\+.*,[]
2,ta\+.*,[]
3,.ks.*,"[60seks, Esimeseks, Igaks, Igaühel, Igaüht, Igaüks, Kelleks, Kelleks, Milleks, Milleks, Milliseks, Mitmesugusteks, Muudeks, Neiks, Neiks, Niisuguseks, SELLEKS, Samaks, Seks, Selleks, Selliseks, Teiseks, Tolleks, esimeseks, igaks, igaühe, igaühega, igaühel, igaühele, igaühes, igaüht, igaühte, igaühte, igaüks, iseendaks, kelleks, kelleks, kinnita-mi-seks, kõigeks, kõigiks, milleks, milleks, millekski, milliseks, mingiks, mingisuguseks, missuguseks, mitmeks, mitmesuguseks, muudeks, muuks, mõlemaks, mõneks, mõningaseks, nendeks, nendeks, niisuguseks, omadeks, omaks, paljudeks, paljuks, samaks, samasuguseks, seks, selleks, selliseks, sellisteks, taoliseks, teiseks, teisteks, tolleks, vormel-üks, Ühe, Ühed, Üheks, Ühel, Ühele, Ühelgi, Ühelt, Ühes, Ühest, Üht, Ühte, Ühte, Ühtedel, Ühtedele, Ühtegi, Ühtegi, Üks, Ükski, Üksteise, Üksteise, Üksteist, ühe, ühe-, ühed, ühega, ühegagi, ühegi, üheks, ühel, ühele, ühelegi, ühelgi, ühelt, üheltki, ühena, ühes, üheski, ühessegi, ühest, üht, üht-, ühte, ühte, ühte-, ühte-, ühtede, ühtedel, ühtedele, ühtegi, ühtegi, ühtesid, ühtki, üks, üks-, ükski, üksteise, üksteise, üksteisega, üksteisel, üksteisele, üksteisest, üksteist]"
4,.ks_teise.*,"[Üksteise, Üksteise, Üksteist, üksteise, üksteise, üksteisega, üksteisel, üksteisele, üksteisest, üksteist]"
5,.kski.*,[millekski]
6,eikeegi.*,[]
7,eimiski.*,[]
8,emb-kumb.*,"[emb-kumb, emba-kumba, emba-kumba]"
9,emb\+.*,[]


### Group the pronouns by lemmas

In [5]:
from collections import defaultdict
pronouns_by_lemmas = defaultdict(set)
file = '../corpora/tokens_analysis.json'
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['partofspeech'] == 'P':
            pronouns_by_lemmas[a['lemma']].add(token)

data = [{'lemma': key, 'examples': sorted(value)} for key, value in sorted(pronouns_by_lemmas.items())]
pandas.DataFrame.from_records(data, columns=['lemma', 'examples'])

,lemma,examples
0,60see,[60seks]
1,F1-sina,[F1-te]
2,Jalta-taoline,[Jalta-taolist]
3,MV-sina,[MV-te]
4,eel-kõik,[eel-kõige]
5,eikeegi,[eikellegi]
6,emb-kumb,"[emb-kumb, emba-kumba]"
7,esimene,"[ESIMENE, ESIMESE, ESIMESED, Esimene, Esimese, Esimesed, Esimeseks, Esimesel, Esimesena, Esimeses, Esimesest, Esimesi, Esimesse, Esimest, Esimeste, Esimestel, esimene, esimese, esimesed, esimeseks, esimesel, esimesele, esimesena, esimeses, esimesest, esimesi, esimesse, esimest, esimeste, esimestel, esimestele, esimestena, esimestes, esimestest]"
8,ettevõt-sina,[ettevõt-teid]
9,hingetõm-mina,[hingetõm-me]


## Morph analysis of tokens *vs* tmorftrtabel.txt

In [6]:
pos_form_pairs = {}
from estnltk.rewriting.syntax_preprocessing.syntax_preprocessing import MorphToSyntaxMorphRewriter
tmorph_file = '../estnltk/estnltk/rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt'
pos_form_pairs['in tmorftrtabel.txt'] = set(MorphToSyntaxMorphRewriter.load_fs_mrf_to_syntax_mrf_translation_rules(tmorph_file))

pos_form_pairs['in corpus'] = set()
for a in analysis_examples:
    pos_form_pairs['in corpus'].add((a[5], a[6]))
    
print('(pos, form) pairs in tmorftrtabel.txt, but not in corpus:')
print(pos_form_pairs['in tmorftrtabel.txt'] - pos_form_pairs['in corpus'])
print('\n(pos, form) pairs in corpus, but not in tmorftrtabel.txt:')
not_in_tmorftrtabel = pos_form_pairs['in corpus'] - pos_form_pairs['in tmorftrtabel.txt']
for pos_form in not_in_tmorftrtabel:
    print('{:<20}'.format(str(pos_form)), end='')
    for a in analysis_examples:
        if pos_form == (a[5], a[6]):
            print(analysis_examples[a], end=', ')
    print()

(pos, form) pairs in tmorftrtabel.txt, but not in corpus:
{('U', 'pl es'), ('O', 'sg ter'), ('V', 'nuvat'), ('V', 'tuvat'), ('Y', 'pl all'), ('P', ''), ('O', 'pl ter'), ('U', 'sg kom'), ('Y', 'pl tr'), ('O', 'sg ab'), ('U', 'pl kom'), ('N', 'sg ab'), ('Y', 'pl abl'), ('U', 'pl ter'), ('U', 'pl ill'), ('Y', 'pl es'), ('Y', 'pl ter'), ('N', 'pl ab'), ('U', 'sg ter'), ('C', 'sg ter'), ('Y', 'sg n'), ('C', 'pl ter'), ('N', 'pl ter'), ('U', 'pl ab'), ('U', 'sg ab'), ('U', 'pl abl'), ('Y', 'pl ill'), ('N', 'pl tr')}

(pos, form) pairs in corpus, but not in tmorftrtabel.txt:
('V', 'neg me')     ärme, Ärme, 
('H', '')           Ülemöödunud, Virisemata, 


## Roots that are in corpus but not in abileksikon06.utf.lx

In [7]:
# find all pairs (root, partofspeech) from abileksikon
abileksikon_file = '../estnltk/estnltk/rewriting/syntax_preprocessing/rules_files/abileksikon06utf.lx'

root_pos = set()
with open(abileksikon_file, mode='r') as f:
    while True:
        root = next(f, None)
        if root is None:
            break
        root = root.rstrip('\n')
        rule = next(f).rstrip('\n')
        if ('_K_' in rule):
            root_pos.add((root, 'K'))
        if ('_V_' in rule):
            root_pos.add((root, 'V'))
        if ('_Y_' in rule):
            root_pos.add((root, 'Y'))
        if ('_Z_' in rule):
            root_pos.add((root, 'Z'))

not_in_abileksikon = {'K': set(), 'V': set(), 'Y': set(), 'Z': set()}

file='../corpora/tokens_analysis.json'
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['partofspeech'] in {'K', 'V'}:
            if (a['root'], a['partofspeech']) not in root_pos:
                not_in_abileksikon[a['partofspeech']].add(a['root'])
        elif a['partofspeech'] in {'Y', 'Z'}:
            if len(a['root']) > 1:
                continue
            if (a['root'], a['partofspeech']) not in root_pos:
                not_in_abileksikon[a['partofspeech']].add(a['root'])

# Y ja Z jaoks me abileksikoni syntax_preprocessingu juures ei kasuta,
# piisab PunctuationTypeRewriter-st.
print('Y (max one character)')
print(not_in_abileksikon['Z'])
print('Z (max one character)')
print(not_in_abileksikon['Z'])
print('K')
print(not_in_abileksikon['K'])
print('V')

print(sorted(not_in_abileksikon['V']))

Y (max one character)
{'', '*', '>', '+', ']', '‹', '›', '[', '’', '=', '<', '/'}
Z (max one character)
{'', '*', '>', '+', ']', '‹', '›', '[', '’', '=', '<', '/'}
K
{'arust', 'jõulu-ajal', 'alates', 'kohaselt', 'ilma+st-ilma', 'maa-rahu-eest', 'peast', 'vastavalt'}
V
['Ameerika+s-käi', 'Eesti-poolne', 'Kreeka-rooma', 'Trondheimi-järgne', 'VIST._ole', 'Vahe_mere-stiili', 'Viru_maa-tuuri', 'aadu', 'aasa', 'aasta-paari', 'abi_ellu', 'able', 'aborti', 'administreeri', 'adopteeri', 'adresseeri', 'aeglusta', 'aeta', 'agiteeri', 'ahenda', 'ahmi', 'aiGu', 'aja_kohasta', 'aja_loo-alane', 'ajasta', 'ajata', 'ajenda', 'akka', 'akrediteeri', 'aktepteeri', 'aktiveeru', 'aktiviseeri', 'aktsentueeri', 'aktsepteeri', 'aktsepti', 'akumuleeri', 'ala_hinda', 'ala_väärista', 'alarmi', 'algata', 'ali', 'all_kirjasta', 'alle', 'alternatiiv-programmi', 'ambu', 'ammenda', 'ammuta', 'amortiseeru', 'analüüsi', 'ankeeti', 'anketeeri', 'anneta', 'annuleeri', 'annulleeri', 'arcane', 'aresti', 'areta', 'arhiivi', 

## Everything that comes afer '=' in root

Group the results by part of speech.

In [8]:
from collections import defaultdict
root_eq_ends = defaultdict(set)
file = '../corpora/tokens_analysis.json'
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if '=' in a['root']:
            root_split = a['root'].split('=')
            if len(root_split) > 2:
                print('more than one "=" in root:', a['root'])
            root_eq_ends[a['partofspeech']].add(root_split[-1])

data = [{'POS': key, 'ends': sorted(value)} for key, value in sorted(root_eq_ends.items())]
pandas.DataFrame.from_records(data, columns=['POS', 'ends'])

more than one "=" in root: mõel=nu+d-tei=nud
more than one "=" in root: mõel=nu+d-tei=nud
more than one "=" in root: mõel=nu+d-tei=nud
more than one "=" in root: sorki=mis-nuusuta=mis_meetod


,POS,ends
0,A,"[dav, dud, istlik, istlik-avangardistlik, ja+te_poolne, ja+te_vaheline, jalik, ke, laslik, lik, lik-optimistlik, lik-poliitiline, line, luslik, mata, matu, mis_himuline, mis_tarkune, mis_vastane, nu+d-kirjutanud, nu+d-sulav, nud, tamatu, tav, tu, tu+d-armastatud, tu+d-kirjutatud, tud, tuslik, tuslik-kasvatuslik, v]"
1,C,"[istlikum, likum, lisem, m, matum, num, tavam, tum, vam]"
2,D,"[istlikult, istlikumalt, konniti, laslikult, likult, likumalt, liselt, lisemalt, lt, malt, matult, mini, nult, numalt, tamatult, tavamalt, ti, tult, valt, vamalt]"
3,H,"[im, ist, istlik, ja, ja+te_koda, ja+te_ühistu, ke, kene, kus, laslik, lik, likkus, likum, line, lisem, m, mata, mine, mis_keskus, mis_klubi, mis_taotlus, nu, sus, tav, tu, tuim, tus, tuslik, v, vaim, vam]"
4,O,[s]
5,S,"[du, dus, ist, ja, ja+te_hulk, ja+te_koda, ja+te_kollektiiv, ja+te_koolitus, ja+te_pere, ja+te_rivi, ja+te_transport, ja+te_vahetus, ja+te_vedu, ja+te_ühistu, ja-lao_hoidja, janna, jaskond, ke, kene, kkus, kus, lane, likkus, lisus, lus, matus, mine, mis, mis_aeg, mis_aparaat, mis_kava, mis_kee, mis_keel, mis_keskus, mis_kiir, mis_kiird, mis_kiirus, mis_kindlus, mis_kinnitus, mis_kleit, mis_klubi, mis_kogu, mis_komisjon, mis_konverents, mis_kord, mis_kulg, mis_kulu, mis_kurk, mis_küsimus, mis_leping, mis_luba, mis_meetod, mis_märgu_anne, mis_nähe, mis_oskus, mis_periood, mis_pjedestaal, mis_plaan, mis_plaat, mis_plats, mis_poliitika, mis_probleem, mis_protokoll, mis_protsess, mis_suund, mis_süsteem, mis_taotlus, mis_tarkus, mis_teenus, mis_tehnoloogia, mis_tingimus, mis_tseremoonia, mis_tulemus, mis_tõend, mis_töö, mis_vahend, mis_vald, mis_vastane, mis_vesi, mis_viis, mis_vorm, mis_võim, mis_võimalus, mis_võime, mis_üles_anne, mus, nu, nu+d-kirjutanu, nu+d-teinu, s, sus, sus_põhi_mõte, tamatus, tavus, tsus, tu, tu+d-armastatu, tu+d-kirjutatu, tus, us, vus]"
6,U,"[duim, im, lisim, nuim, tavaim, tuim, vaim]"
7,V,"[nu+d-kirjuta, nu+d-sula, nu+d-tege, tu+d-armasta, tu+d-kirjuta]"
8,Z,[]


## All tokens that have part of speech "V" and form "neg"

In [9]:
file = '../corpora/tokens_analysis.json'
aux_neg_tokens = []
for token, analysis in yield_tokens_analysis(file):
    for a in analysis:
        if a['form'] == 'neg' and a['partofspeech'] == 'V':
            aux_neg_tokens.append(token)
            break
aux_neg_tokens

['EI', 'Ei', 'ei']